<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Pre_training_5M_Corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_25M_10_000_condensed"
dataset = load_dataset(dataset_name)
print(dataset)
print_gpu_utilization()

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

In [ ]:
adapter_hub_name = "domain_adapter_test"

adapter_name = "domain_training_test"
adapter_type = "seq_bn" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")
print_gpu_utilization()

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

# Train

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_pretraining_5M_model_corrected",
    learning_rate=0.0001, # USING ADAPTER RECOMENDATION
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32, 
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=1, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    # lr_scheduler_type="linear",
    # evaluation_strategy="steps",
    # eval_steps=500, # Evaluates every 50 steps
    # save_strategy="steps",
    # save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    # save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    # push_to_hub=True,
    # hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
)

In [ ]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

In [ ]:
eval_results = trainer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
results = trainer.train()
print_gpu_utilization()
print(results)

In [ ]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()